In [1]:
using Revise
using DifferentialEquations
using Flux
using DiffEqFlux
using Optim
using Plots
using FileIO
using JLD2
using OffsetArrays

include("/mnt/d/Coding/Github/Kinetic.jl/src/Kinetic.jl")
using .Kinetic

In [2]:
function bgk!(df, f, p, t)
    g, tau = p
    df .= (g .- f) ./ tau
end

bgk! (generic function with 1 method)

In [3]:
D = read_dict("../code/shock/shock1D.txt")
for key in keys(D)
    s = Symbol(key)
    @eval $s = $(D[key])
end

case = shock
space = 1d1f1v
nSpecies = 1
interpOrder = 1
limiter = vanleer
cfl = 0.95
maxTime = 250.0
x0 = -25.0
x1 = 25.0
nx = 50
pMeshType = uniform
nxg = 0
umin = -10.0
umax = 10.0
nu = 36
nug = 0
vMeshType = rectangle
knudsen = 1.0
mach = 3.0
prandtl = 1.0
inK = 0.0
omega = 0.5
alphaRef = 1.0
omegaRef = 0.5
tLen = 3
nh = 12



In [4]:
γ = 3.
set = Setup(case, space, nSpecies, interpOrder, limiter, cfl, maxTime)
pSpace = PSpace1D(x0, x1, nx, pMeshType, nxg)
μᵣ = ref_vhs_vis(knudsen, alphaRef, omegaRef)
gas = GasProperty(knudsen, mach, prandtl, inK, γ, omega, alphaRef, omegaRef, μᵣ)
vSpace = VSpace1D(umin, umax, nu, vMeshType, nug)
wL, primL, fL, bcL, wR, primR, fR, bcR = ib_rh(mach, γ, vSpace.u)
ib = IB1D1F(wL, primL, fL, bcL, wR, primR, fR, bcR)

ks = SolverSet(set, pSpace, vSpace, gas, ib, pwd());

In [5]:
ctr = OffsetArray{ControlVolume1D1F}(undef, eachindex(ks.pSpace.x))
face = Array{Interface1D1F}(undef, ks.pSpace.nx+1)
for i in eachindex(ctr)
    if i <= ks.pSpace.nx÷2
        ctr[i] = ControlVolume1D1F(ks.pSpace.x[i], ks.pSpace.dx[i], Float32.(ks.ib.wL), Float32.(ks.ib.primL), Float32.(ks.ib.fL))
    else
        ctr[i] = ControlVolume1D1F(ks.pSpace.x[i], ks.pSpace.dx[i], Float32.(ks.ib.wR), Float32.(ks.ib.primR), Float32.(ks.ib.fR))
    end
end
for i=1:ks.pSpace.nx+1
    face[i] = Interface1D1F(ks.ib.fL)
end

In [6]:
sos = sound_speed(ks.ib.primR, γ)
vmax = ks.vSpace.u1 + sos
tmax = vmax / ks.pSpace.dx[1]
dt = Float32(ks.set.cfl / tmax)
tSpan = (0.f0, dt)
tRan = range(tSpan[1], tSpan[2], length=tLen)

0.0f0:0.035737388f0:0.071474776f0

In [7]:
residual = Array{Float32}(undef, 3)
for iter=1:1000
    Kinetic.evolve!(ks, ctr, face, dt)
    Kinetic.update!(ks, ctr, face, dt, residual)
end

In [8]:
data_init = [ Float32.(maxwellian(vSpace.u, ib.primL)) |> Array, Float32.(maxwellian(vSpace.u, ib.primR)) |> Array ]
data_para = [ [maxwellian(ks.vSpace.u, ks.ib.primL), vhs_collision_time(ks.ib.primL, ks.gas.μᵣ, ks.gas.ω)],
              [maxwellian(ks.vSpace.u, ks.ib.primR), vhs_collision_time(ks.ib.primR, ks.gas.μᵣ, ks.gas.ω)] ];

data_init = []
data_para = []
for i=2:49
    push!(data_init, ctr[i].f[1:end])
    push!(data_para, [maxwellian(ks.vSpace.u, ctr[i].prim), vhs_collision_time(ctr[i].prim, ks.gas.μᵣ, ks.gas.ω)])
end

In [9]:
prob = ODEProblem(bgk!, data_init[1], tSpan, data_para[1])
data_boltz1 = solve(prob, Tsit5(), saveat=tRan) |> Array;

prob = ODEProblem(bgk!, data_init[2], tSpan, data_para[2])
data_boltz2 = solve(prob, Tsit5(), saveat=tRan) |> Array;

data_boltz = [data_boltz1, data_boltz2];

data_boltz = []
for i in eachindex(data_init)
    prob = ODEProblem(bgk!, data_init[i], tSpan, data_para[i])
    push!(data_boltz, solve(prob, Tsit5(), saveat=tRan) |> Array)
end

In [10]:
dudt = FastChain( (x, p) -> x.^3,
                   FastDense(vSpace.nu, vSpace.nu*10, tanh),
                   FastDense(vSpace.nu*10, vSpace.nu*10, tanh),
                   FastDense(vSpace.nu*10, vSpace.nu) )
n_ode = NeuralODE(dudt, tSpan, Tsit5(), saveat=tRan)

(::NeuralODE{FastChain{Tuple{var"#3#4",FastDense{typeof(tanh),DiffEqFlux.var"#initial_params#75"{typeof(Flux.glorot_uniform),typeof(Flux.zeros),Int64,Int64}},FastDense{typeof(tanh),DiffEqFlux.var"#initial_params#75"{typeof(Flux.glorot_uniform),typeof(Flux.zeros),Int64,Int64}},FastDense{typeof(identity),DiffEqFlux.var"#initial_params#75"{typeof(Flux.glorot_uniform),typeof(Flux.zeros),Int64,Int64}}}},Array{Float32,1},Nothing,Tuple{Float32,Float32},Tsit5,Tuple{},Base.Iterators.Pairs{Symbol,StepRangeLen{Float32,Float64,Float64},Tuple{Symbol},NamedTuple{(:saveat,),Tuple{StepRangeLen{Float32,Float64,Float64}}}}}) (generic function with 4 methods)

In [11]:
function loss_n_ode(p)
    pred = n_ode(data_init[1], p) |> Array

    loss = 0.
    for i in eachindex(data_init)
        loss += sum(abs2, (n_ode(data_init[i], p) |> Array) .- data_boltz[i])
    end
    
    return loss, pred
end

loss_n_ode (generic function with 1 method)

In [12]:
cb = function (p, l, pred; doplot=false)
    display(l)
    # plot current prediction against dataset
    if doplot
        pl = plot(tRan, data_boltz[1][vSpace.nu÷2,:], lw=2, label="Exact")
        scatter!(pl, tRan, pred[vSpace.nu÷2,:], lw=2, label="NN")
        display(plot(pl))
    end
    return false
end

#5 (generic function with 1 method)

In [ ]:
res = DiffEqFlux.sciml_train(loss_n_ode, n_ode.p, ADAM(0.002, (0.9, 0.95)), cb=cb, maxiters=200)

0.00411676628573332

loss: 0.00412:   0%|▎                                        |  ETA: 1:53:13

0.009690108876384329

loss: 0.00969:   1%|▍                                        |  ETA: 1:04:21

0.004083866313521867

...

loss: 5.68e-05:  91%|█████████████████████████████████████▎   |  ETA: 0:01:09

6.945833860072526e-5

loss: 6.95e-05:  92%|█████████████████████████████████████▌   |  ETA: 0:01:05

7.246051598031045e-5

loss: 7.25e-05:  92%|█████████████████████████████████████▊   |  ETA: 0:01:01